In [ ]:
# Import necessary modules
import boto3
import json
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import sys
import time

from concurrent.futures import ThreadPoolExecutor, as_completed

here = os.path.dirname(os.path.abspath("__file__"))
project_root = os.path.abspath(os.path.join(here, '../../'))
sys.path.insert(0, project_root)

from smartfarm.core.mpc.mpc import MPC
from core.mpc.mpc_params import MPCParams
from core.mpc.mpc_bounds import ControlInputBounds

from core.model.model_carrying_capacities import ModelCarryingCapacities
from core.model.model_disturbances import ModelDisturbances
from core.model.model_growth_rates import ModelGrowthRates
from core.model.model_initial_conditions import ModelInitialConditions
from core.model.model_params import ModelParams
from core.model.model_typical_disturbances import ModelTypicalDisturbances
from core.model.model_sensitivities import ModelSensitivities

from core.model.model_helpers import get_sim_inputs_from_hourly

from core.plotting.plotting import *

## Model Parameters

In [ ]:
# Set ModelParams (time stepping)
dt               = 1    # hours/step
simulation_hours = 2900 # hours
closed_form      = False
verbose          = False

## Input Disturbances by Time Step

In [8]:
# Hourly precipitation, radiation, and temperature from CSV
hourly_disturbances = pd.read_csv(
    '../../io/inputs/hourly_prcp_rad_temp_iowa.csv'
)

In [ ]:
precipitation = get_sim_inputs_from_hourly(
    hourly_array     = 0.5*hourly_disturbances['Hourly Precipitation (in)'].to_numpy(), # imagine it's a drought year,
    dt               = dt,
    simulation_hours = simulation_hours,
    mode             = 'split')
temperature = get_sim_inputs_from_hourly(
    hourly_array     = hourly_disturbances['Temperature (C)'].to_numpy(),
    dt               = dt,
    simulation_hours = simulation_hours,
    mode             = 'split')
radiation = get_sim_inputs_from_hourly(
    hourly_array     = hourly_disturbances['Hourly Radiation (W/m2)'].to_numpy(),
    dt               = dt,
    simulation_hours = simulation_hours,
    mode             = 'split')

## Run parameter sweeps with AWS Lambda

In [ ]:
_LAMBDA_FUNCTION_NAME = "smartfarm-mpc-eval"
_LAMBDA_REGION_NAME   = "us-west-2"

lambda_client = boto3.client(
    "lambda",
    region_name=_LAMBDA_REGION_NAME,
)

In [ ]:
weight_irrigation_options                     = [0.01, 0.1, 1.0, 10]
weight_fertilizer_options                     = [0.00001, 0.0001, 0.001, 0.01, 0.1, 1.0]
weight_fruit_biomass_options                  = [1.0, 10.0, 100.0, 1000.0]
weight_cumulative_average_water_options       = [1.0]
weight_cumulative_average_fertilizer_options  = [1.0]
weight_cumulative_average_temperature_options = [1.0]
weight_cumulative_average_radiation_options   = [1.0]

In [14]:
# Serialize all weight combinations to dict of lists
weights = []
for wI in weight_irrigation_options:
    for wF in weight_fertilizer_options:
        for wB in weight_fruit_biomass_options:
            for wCAW in weight_cumulative_average_water_options:
                for wCAF in weight_cumulative_average_fertilizer_options:
                    for wCAT in weight_cumulative_average_temperature_options:
                        for wCAR in weight_cumulative_average_radiation_options:
                            weights.append(
                                {
                                    "weight_irrigation":                     wI,
                                    "weight_fertilizer":                      wF,
                                    "weight_fruit_biomass":                   wB,
                                    "weight_cumulative_average_water":        wCAW,
                                    "weight_cumulative_average_fertilizer":   wCAF,
                                    "weight_cumulative_average_temperature":  wCAT,
                                    "weight_cumulative_average_radiation":    wCAR,
                                }
                            )

In [15]:
len(weights)

96

In [ ]:
# Build shared context once
total_time_steps = int(simulation_hours / dt)
context = {
    # Time stepping / horizon
    "dt": float(dt),
    "total_time_steps": int(total_time_steps),
    "simulation_hours": int(simulation_hours),
    "closed_form":      bool(closed_form),

    # Disturbances (time step)
    "precipitation": np.asarray(precipitation, dtype=float).tolist(),
    "temperature":   np.asarray(temperature,   dtype=float).tolist(),
    "radiation":     np.asarray(radiation,     dtype=float).tolist(),
}

In [17]:
# Prepare batches as (start, end) index pairs
batch_size  = 1
batches = [
    (start, min(start + batch_size, len(weights)))
    for start in range(0, len(weights), batch_size)
]

In [18]:
def invoke_batch(start: int, end: int):
    """Invoke Lambda for members[start:end] and return (start, end, costs_array)."""
    batch = weights[start:end]
    payload = {
        "weights": batch,
        "context": context,
    }

    t0 = time.time()
    response = lambda_client.invoke(
        FunctionName=_LAMBDA_FUNCTION_NAME,
        InvocationType="RequestResponse",
        Payload=json.dumps(payload).encode("utf-8"),
    )
    t1 = time.time()

    raw_payload = response["Payload"].read().decode("utf-8")
    result = json.loads(raw_payload)

    body = result.get("body", result)
    if isinstance(body, str):
        body = json.loads(body)

    batch_output = np.array(body["output"], dtype=float)

    if batch_output.shape[0] != (end - start):
        raise ValueError(
            f"Lambda returned {batch_output.shape[0]} costs for a batch "
            f"of size {end - start} (start={start}, end={end})."
        )

    return start, end, batch_output

In [19]:
# Fire batches in parallel
max_workers = len(batches)
results = np.zeros(len(batches), dtype=float)

with ThreadPoolExecutor(max_workers=max_workers) as executor:
    futures = [
        executor.submit(invoke_batch, start, end)
        for (start, end) in batches
    ]

    for fut in as_completed(futures):
        start, end, batch_output = fut.result()
        results[start:end] = batch_output

KeyError: 'output'